# 1/ OLIVIER_IT ⋈ 6SIGMA (It_equipment) ⋈ CMDB

Vérification de la présence des equipements Olivier_it simultanément sur 6Sigma et sur la CMDB :
    
    - Controle de l'asset_id
    - Controle de Position 
    - COntrole de la hauteur

## Provient d'Olivier_it_Workflow

`IT_Equipment_v2` : Table IT_Equipment provenant d'un fichier csv

In [ ]:
DROP TABLE IF EXISTS `IT_Equipment_v2` ;
CREATE TABLE IT_Equipment_v2
as
SELECT * FROM `IT_Equipment`

`assets_filtree_v2` : Table assets_new (CMDB) filtré aux deux localisations : FRVLB0C2 / FRTCRAVA

In [ ]:
DROP TABLE IF EXISTS `assets_filtree_v2` ;
CREATE TABLE assets_filtree_v2
as
select *
from `assets_new`
where (API = 'FRVLB0C2' or API = 'FRTCRAVA') ;

In [ ]:
ALTER TABLE assets_filtree_v2
ADD COLUMN Rack varchar(255),
ADD COLUMN Slot varchar(255),
ADD COLUMN Slot_begin varchar(255),
ADD COLUMN Slot_end varchar(255);

In [ ]:
UPDATE assets_filtree_v2
SET 
	Rack = SUBSTRING_INDEX(DALLE, '/', 1)  ,
	Slot = SUBSTRING_INDEX(DALLE, '/', -1) ,
	Slot_begin = SUBSTRING_INDEX(SUBSTRING_INDEX(DALLE, '/', -1), '-', 1) ,
	Slot_end = SUBSTRING_INDEX(DALLE, '-', -1)

In [ ]:
DROP TABLE IF EXISTS `assets_filtree_reduced` ;
CREATE TABLE assets_filtree_reduced
AS
SELECT ype, subtype, ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End, No_serie, etat, api, tag_partenaire
from `assets_filtree_v2` aa
WHERE SALLE in ('P1', 'P2', 'P3', 'P4', '0')

In [ ]:
DROP TABLE IF EXISTS `assets_filtree_reduced_not_retired` ;
CREATE TABLE assets_filtree_reduced_not_retired
AS
SELECT ype, subtype, ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End, No_serie, etat, api, tag_partenaire
from `assets_filtree_v2` aa
WHERE SALLE in ('P1', 'P2', 'P3', 'P4')

##### On ajoute la même forme de position que sur 6sigma et olivier_it pour fluidifier les query

In [ ]:

ALTER TABLE `assets_filtree_reduced_not_retired`
ADD COLUMN reperage_id varchar(255) ;

update `assets_filtree_reduced_not_retired`
set reperage_id = concat(Rack,":Slot ", slot_begin)


`IT_Equipment_match_position_reduced_v3` : groupement des lignes par positions et affichage d'un seul asset_id par positions + nombres d'equipements sur la même position

In [ ]:
DROP VIEW IF EXISTS IT_Equipment_match_position_reduced_v3_not_retired;
CREATE VIEW IT_Equipment_match_position_reduced_v3_not_retired
as
SELECT it.nom_de_la_salle, it.reperage_id, count(it.reperage_id) as RowNumber,
		concat(aa.asset_id) asset_id

FROM   `olivier_it` it  
INNER JOIN `assets_filtree_reduced_not_retired`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.reperage_id = aa.reperage_id
)  
GROUP BY it.reperage_id

## Equipement non présent sur Olivier_IT

#### Equipement 6sigma non présent sur Olivier_it sur la base de la position (salle, rack, position de départ)

`olivier_it_eq_not_in_6sigma` : Liste des équipements 6sigma non présent sur Olivier_it
`olivier_it_eq_not_in_CMDB` : Liste des équipements CMDB non présent sur Olivier_it

In [ ]:
DROP TABLE IF EXISTS olivier_it_eq_not_in_6sigma ;
CREATE TABLE olivier_it_eq_not_in_6sigma
as 
select 
it2.modele, it2.type, it2.fabricant, it2.nom, it2.short_name, it2.serial_number, it2.asset_id,
it2.ASSET_TAG, it2.ASSET_TAG Asset_Number, it2.Nom_de_la_salle, it2.Reperage_ID Reperage_ID_origin, it2.Reperage_ID, it2.End_Move_Target, null null_1 ,null null_2, null null_3, null null_4, null null_5
from `olivier_it` it
RIGHT JOIN `IT_Equipment` it2 
ON it.reperage_id = it2.reperage_id
and it.nom_de_la_salle = it2.nom_de_la_salle

where it.reperage_id is null
order by reperage_id asc

In [ ]:
DROP TABLE IF EXISTS olivier_it_eq_not_in_cmdb ;
CREATE TABLE olivier_it_eq_not_in_cmdb
as 
select aa.MODEL, aa.subtype, aa.FABRICANT, aa.NOM_LONG, aa.No_serie, aa.No_serie Serial_number, aa.ASSET_ID,
aa.tag_partenaire asset_tag, aa.tag_partenaire asset_number, aa.SALLE, aa.reperage_id reperage_id_origin, 
aa.reperage_id, NULL end_move_target, null null_1 ,null null_2, null null_3, null null_4, null null_5
from `olivier_it` it
RIGHT JOIN `assets_filtree_reduced_not_retired` aa 
ON it.reperage_id = aa.reperage_id
and it.nom_de_la_salle = aa.salle

where it.reperage_id is null and aa.dalle != '' and etat != 'Retired' 
order by reperage_id asc

## On ajoute les équipements manquants à olivier_it

ajout d'un attribut `from_sources` qui contient la provenance de l'équipement :

    - 6Sigma
    - CMDB
    - Olivier_it 

In [ ]:
alter table olivier_it
ADD COLUMN from_source VARCHAR(255) ;

#### ADD ROW

In [ ]:
INSERT INTO `olivier_it` 
SELECT it.*, "6SIGMA" FROM `olivier_it_eq_not_in_6sigma` it

In [ ]:
INSERT INTO `olivier_it` 
SELECT it.*, "CMDB" FROM `olivier_it_eq_not_in_cmdb` it

In [ ]:
update `olivier_it`
set from_source = 'Olivier_it'
where from_source is null

In [ ]:
ALTER TABLE olivier_it 
ADD COLUMN status_6Sigma varchar(255),
ADD COLUMN status_CMDB varchar(255),
ADD COLUMN status_position_6Sigma varchar(255),
ADD COLUMN status_position_CMDB varchar(255),
ADD COLUMN status_hauteur varchar(255);

## Status d'asset_id et de position

In [ ]:

UPDATE `olivier_it` it 
INNER JOIN `IT_Equipment_records` it2 ON it.ASSET_ID = it2.ASSET_ID 

SET it.status_6Sigma = 'OK'

WHERE (it.ASSET_ID != '' or it.ASSET_ID != '' or it2.ASSET_ID != '' or it2.ASSET_ID != '') and it2.INSERTED = (select * from `last_date_insertion`)

In [ ]:

UPDATE `olivier_it` it 
INNER JOIN `assets_filtree_reduced_not_retired` it2 ON it.ASSET_ID = it2.ASSET_ID 

SET it.status_cmdb = 'OK'

WHERE (it.ASSET_ID != '' or it.ASSET_ID != '' or it2.ASSET_ID != '' or it2.ASSET_ID != '')

In [ ]:

UPDATE `olivier_it` it 
SET status_6Sigma = 'KO'
WHERE status_6Sigma is null ;



In [ ]:
UPDATE `olivier_it` it 
SET status_CMDB = 'KO'
WHERE status_CMDB is null

SELECT * FROM `IT_Equipment_match_position_reduced_v3` it2

INNER JOIN `olivier_it` it 
ON it.reperage_id = it2.reperage_id
and it.nom_de_la_salle = it2.nom_de_la_salle

In [ ]:
-- 1823
UPDATE `olivier_it` it
INNER JOIN `IT_Equipment` it2 
ON it.reperage_id = it2.reperage_id
and it.nom_de_la_salle = it2.nom_de_la_salle

SET it.status_position_6Sigma = "OK"

WHERE  (
   it2.modele not like '%BL460C%'
OR it2.modele not like '%BL660C%'
OR it2.modele not like '%BL20P%'
OR it2.modele not like '%BL860C%'
OR it2.modele not like '%BL420C%'
OR it2.modele not like '%BL680C%'
OR it2.modele not like '%WS460C%'
OR it2.modele not like '%BL870C%' )

In [ ]:

UPDATE `olivier_it` it 
INNER JOIN `assets_filtree_reduced_not_retired`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.reperage_id = aa.reperage_id
)  
    
INNER JOIN `IT_Equipment_match_position_reduced_v3_not_retired` it2
ON it.reperage_id = it2.reperage_id
and it.nom_de_la_salle = it2.nom_de_la_salle

SET it.status_position_CMDB = 'OK'

WHERE  (
   aa.model not like '%BL460C%'
OR aa.model not like '%BL660C%'
OR aa.model not like '%BL20P%'
OR aa.model not like '%BL860C%'
OR aa.model not like '%BL420C%'
OR aa.model not like '%BL680C%'
OR aa.model not like '%WS460C%'
OR aa.model not like '%BL870C%' )

In [ ]:

UPDATE `olivier_it` it 
SET status_position_6Sigma = 'KO'
WHERE status_position_6Sigma is null ;


UPDATE `olivier_it` it 
SET status_position_CMDB = 'KO'
WHERE status_position_CMDB is null